In [1]:
#pip install kafka

     |████████████████████████████████| 207 kB 7.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
#pip install pymongo

     |████████████████████████████████| 479 kB 6.5 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
#pip install kafka-python

     |████████████████████████████████| 246 kB 5.9 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
#pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [1]:
from kafka import KafkaConsumer
import json
import pandas as pd
from pymongo import MongoClient
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import udf, col, lower, regexp_replace
from pyspark.ml.feature import Tokenizer, StopWordsRemover, NGram, HashingTF, IDF
from pyspark.ml import Pipeline
from pyspark.ml import PipelineModel
from pyspark.sql.types import StructType,StructField
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import LogisticRegressionModel

import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')

# Connection à Kafka & streaming 

In [ ]:
# Connection à MongoDB
client = MongoClient('mongo',27017)

# Ouverture de la Session Spark
spark = SparkSession.\
builder.\
appName("myApp").\
config("spark.mongodb.input.uri", "mongodb://mongo:27017/db_Booking.*").\
config("spark.mongodb.output.uri", "mongodb://mongo:27017/db_Booking.*").\
config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.0").\
getOrCreate()

#config("spark.driver.memory","5g").\

# Connexion au topic Kafka
post = KafkaConsumer("review",bootstrap_servers=['kafka:9093'], api_version=(2,6,0)) 

# Chargement du pipeline de preprocessing des données
mypipeline =  PipelineModel.load("MyPipeline")

# Chargement de notre modele
mylrModel =  LogisticRegressionModel.load("mylrModel")

schema = StructType([
    StructField('c0', IntegerType(),True),
    StructField('hotel_name', StringType(),True),
    StructField('lat', DoubleType(),True),
    StructField('long', DoubleType(),True),
    StructField('average_score', FloatType(),True),
    StructField('review', StringType(),True),
    StructField('polarity', IntegerType(),True),
    StructField('word_counts', IntegerType(),True),
    StructField('tags', StringType(),True),
])


# Streaming et prediction 
for msg in post :
    data=[json.loads(msg.value)]
    review = spark.createDataFrame(data,schema=schema)
    df_prepared= mypipeline.transform(review)
    lrPreds= mylrModel.transform(df_prepared)
    #lrPreds.show(vertical=True)

    lrPreds.select('hotel_name','lat','long','review','word_counts','polarity','prediction','tags').write.format("mongo").mode("append").option("uri","mongodb://mongo:27017/Booking.Review").save()
    lrPreds.select('hotel_name','polarity','prediction').show()


+--------------------+--------+----------+
|          hotel_name|polarity|prediction|
+--------------------+--------+----------+
|The Nadler Kensin...|       0|       0.0|
+--------------------+--------+----------+

+--------------------+--------+----------+
|          hotel_name|polarity|prediction|
+--------------------+--------+----------+
|Radisson Blu Edwa...|       1|       1.0|
+--------------------+--------+----------+

+--------------------+--------+----------+
|          hotel_name|polarity|prediction|
+--------------------+--------+----------+
|M by Montcalm Sho...|       0|       0.0|
+--------------------+--------+----------+

+--------------------+--------+----------+
|          hotel_name|polarity|prediction|
+--------------------+--------+----------+
|Golden Tulip Wash...|       0|       0.0|
+--------------------+--------+----------+

+--------------------+--------+----------+
|          hotel_name|polarity|prediction|
+--------------------+--------+----------+
|The Pa

+--------------------+--------+----------+
|          hotel_name|polarity|prediction|
+--------------------+--------+----------+
|NH Collection Bar...|       0|       0.0|
+--------------------+--------+----------+

+--------------------+--------+----------+
|          hotel_name|polarity|prediction|
+--------------------+--------+----------+
|Radisson Blu Edwa...|       0|       0.0|
+--------------------+--------+----------+

+--------------------+--------+----------+
|          hotel_name|polarity|prediction|
+--------------------+--------+----------+
|DoubleTree by Hil...|       0|       0.0|
+--------------------+--------+----------+

+--------------------+--------+----------+
|          hotel_name|polarity|prediction|
+--------------------+--------+----------+
|Grand Hotel Saint...|       0|       0.0|
+--------------------+--------+----------+

+--------------------+--------+----------+
|          hotel_name|polarity|prediction|
+--------------------+--------+----------+
|NH Col

+--------------------+--------+----------+
|          hotel_name|polarity|prediction|
+--------------------+--------+----------+
|Duquesa Suites Ba...|       1|       1.0|
+--------------------+--------+----------+

+------------+--------+----------+
|  hotel_name|polarity|prediction|
+------------+--------+----------+
|H tel Powers|       1|       1.0|
+------------+--------+----------+

+--------------------+--------+----------+
|          hotel_name|polarity|prediction|
+--------------------+--------+----------+
|The RE London Sho...|       0|       0.0|
+--------------------+--------+----------+

+--------------------+--------+----------+
|          hotel_name|polarity|prediction|
+--------------------+--------+----------+
|Grand Royale Lond...|       1|       1.0|
+--------------------+--------+----------+

+--------------------+--------+----------+
|          hotel_name|polarity|prediction|
+--------------------+--------+----------+
|Hotel Serhs Rivol...|       0|       0.0|
+---

+--------------------+--------+----------+
|          hotel_name|polarity|prediction|
+--------------------+--------+----------+
|Hilton London Met...|       1|       0.0|
+--------------------+--------+----------+

+----------+--------+----------+
|hotel_name|polarity|prediction|
+----------+--------+----------+
|Park Hotel|       1|       1.0|
+----------+--------+----------+

+--------------+--------+----------+
|    hotel_name|polarity|prediction|
+--------------+--------+----------+
|Hotel Da Vinci|       1|       1.0|
+--------------+--------+----------+

+--------------------+--------+----------+
|          hotel_name|polarity|prediction|
+--------------------+--------+----------+
|Catalonia Barcelo...|       0|       0.0|
+--------------------+--------+----------+

+--------------------+--------+----------+
|          hotel_name|polarity|prediction|
+--------------------+--------+----------+
|Radisson Blu Cham...|       0|       0.0|
+--------------------+--------+----------+

